# fast estimation of particle properties
Tim Tyree<br>
9.29.2021<br>
`conda activate pyenv_ub`

__GOAL__: efficiently generate an inline rendering of dRdt versus 1/R and MSD versus tau

In [1]:
from lib.my_initialization import *
from lib import *
from lib.measure.unwrap_and_smooth_cu import *
from lib.routines.unwrap_and_smooth_trajectories_cu import *

#magic    
%autocall 1
%load_ext autoreload
%autoreload 2

import cupy as cp, cudf, numba.cuda as cuda
import rmm
# Switch to RMM allocator
cp.cuda.set_allocator(rmm.rmm_cupy_allocator)

Automatic calling is: Smart


In [2]:
darkmode=False
if darkmode:
    # For darkmode plots
    from jupyterthemes import jtplot
    jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

# (optional) postprocess a token trajectory file

In [3]:
# input_fn=search_for_file()

# # #token FK at DT=0.4
# input_fn="/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-fk-200x200/param_set_8_ds_5.0_tmax_10_diffCoef_0.0005/Log/ic200x200.0.2_traj_sr_400_mem_0.csv"

# # #tokenLR at DT=0.5
# input_fn=f"/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_5.4_diffCoef_0.0005_dt_0.5/trajectories/ic002.31_traj_sr_600_mem_0.csv"
input_fn=f"/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_5.4_diffCoef_0.0005_dt_0.5/trajectories/ic002.11_traj_sr_600_mem_0.csv"

df=pd.read_csv(input_fn)
DT=np.around(get_DT(df),5);print(f"DT={DT}")

DT=0.5


In [4]:
#test routine on one file
testing=True
if testing:
    # ds=5 #cm
    # DS=ds/width
    tavg1=4 #moving average window, in ms
    width=200
    height=width
    use_drop_shorter_than=True
    drop_shorter_than=50 #ms
    tmin=100.#ms
    pid_col='particle'
    t_col='t'
    printing=False
    navg1=int(tavg1/DT)
    save_dir=return_moving_average_of_pbc_trajectories_and_save(
            input_fn, tavg1, pid_col, t_col, DT, width, height,
            use_drop_shorter_than, drop_shorter_than, tmin, printing)
    print(save_dir)

    df=load_smoothed_trajectories(save_dir,pid_col,t_col)
    df.head()

/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_5.4_diffCoef_0.0005_dt_0.5/smoothed_trajectories_navg_8/ic002.11_traj_sr_600_mem_0_smoothed.csv


In [5]:
#run the routine on a whole folder
# print((DT, input_fn))
# save_dir_lst=routine_postprocess_trajectory_folder(input_fn,DT,tavg1=4, npartitions=None,
#                                         width=200,
#                                         height=200,
#                                         use_drop_shorter_than=True,
#                                         drop_shorter_than=50, #ms
#                                         tmin=100., #ms
#                                         pid_col='particle',
#                                         t_col='t',
#                                         printing=False)

# TODO: compute dRdt versus 1/R for unconstrained random samples from smoothed and validated trajectories

In [10]:
# pd.read_csv(save_dir).head()
df.head()

,index,x,y,t,particle,event_id_int,dx_unwrap,dy_unwrap,event_id,x_unwrap,y_unwrap,incol
0,97,88.030335,73.765591,123.0,6,1002116000,0.0,0.0,1.002116e+09,88.030335,73.765591,88.030335
1,105,89.212899,73.010629,123.5,6,1002116000,0.0,0.0,1.002116e+09,89.212899,73.010629,89.212899
2,109,89.852546,72.697096,124.0,6,1002116000,0.0,0.0,1.002116e+09,89.852546,72.697096,89.852546
3,113,91.008877,72.039739,124.5,6,1002116000,0.0,0.0,1.002116e+09,91.008877,72.039739,91.008877
4,114,91.562092,71.769575,125.0,6,1002116000,0.0,0.0,1.002116e+09,91.562092,71.769575,91.562092


In [ ]:
#TODO:compute the distance in x between any points where two spiral tips occur at the same time in t


In [7]:
#TODO(IMMEDIATE FOLLOWUP GOAL): show how insensitive a is to choice in navg1 (and/or navg2 if I'm using it!)
#GOAL_QUESTION: does there exist a tavg1 that produces the right expected a for a single termination event??

In [8]:
# #TODO(later): compute squared displacements
# grouped=df.groupby(pid_col)
# #HINT: squared displacements of particles is result
#  throws AttributeError: DataFrameGroupBy object has no attribute first 
# result = (grouped[['x','y']]-grouped[['x','y']].first())**2

In [9]:
#TODO(later): implement a one_step method on an element in a finite element simulation
# HINT: cudf.Grouper?
# HINT: https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#grouping-with-a-grouper-specification
# HINT: https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#transformation